In [1]:
# Imports
from snowexsql.data import PointData, SingleLocationData, Measurement, Base
from snowexsql.api import PointMeasurements
from sqlalchemy import Column, Date, DateTime, Float, Integer, String, Time, Table
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from typing import List

## Extend the data model
 * Make a new data model that uses the new `points_prototype` table with the copied over `Points` information

In [2]:
# points_prototype


class PointDataOriginal(SingleLocationData, Measurement, Base):
    """
    Class representing the points table. This table holds all point data.
    Here a single data entry is a single coordinate pair with a single value
    e.g. snow depths
    """
    __tablename__ = 'points_original'
    __table_args__ = {"schema": "public"}

    version_number = Column(Integer)
    equipment = Column(String(50))
    value = Column(Float)

    __mapper_args__ = {
        'polymorphic_identity': 'Points',
    }




## Make a new api class
 * Extend the API class and use the new points prototype data model 

In [3]:
class PointsMeasurementsOriginal(PointMeasurements):
    """
    Extend the points api class for the new data model
    """
    DB_NAME = "snow:hackweek@35.86.115.212/snowex"
    MODEL = PointDataOriginal
    

## Use our new api class

In [ ]:
PointsMeasurementsOriginal().all_instruments

## make a new table class that includes foreign keys

In [4]:
# points_prototype
from sqlalchemy import Column, Date, DateTime, Float, Integer, String, Time, ForeignKey
from sqlalchemy.orm import relationship, sessionmaker


# Create a association table for the many-to-many relationship
point_observers = Table(
    "point_observers",
    Base.metadata,
    Column("point_id", ForeignKey("points_prototype.id"), primary_key=True),
    Column("observer_id", ForeignKey("observers.id"), primary_key=True),
    extend_existing=True
)


# Create an Observer table class
class Observer(Base):
    __tablename__ = 'observers'
    __table_args__ = {"schema": "public"}
     # this seems important
    __mapper_args__ = {
        'polymorphic_identity': 'Observer',
    }
    # id is mapped column for many-to-many 
    id: Mapped[int] = mapped_column(primary_key=True)
    # Name of the instrument
    first_name = Column(String(255))
    last_name = Column(String(255))
    email = Column(String(255))


# Create an instruments table class
class Instument(Base):
    __tablename__ = 'instrument'
    __table_args__ = {"schema": "public"}
     # this seems important
    __mapper_args__ = {
        'polymorphic_identity': 'Instrument',
    }
    # auto created id
    id = Column(Integer, primary_key=True)
    # Name of the instrument
    name = Column(String(50), nullable=False)

    # Link each table that use instrument (bi-directional)
    # points = relationship('PointDataPrototype', back_populates='instrument')


# Create a new points table class that links to 
class PointDataPrototype(SingleLocationData, Base):
    __tablename__ = 'points_prototype'
    __table_args__ = {"schema": "public"}
    # this seems important
    __mapper_args__ = {
        'polymorphic_identity': 'Points',
    }

    version_number = Column(Integer)
    equipment = Column(String(50))
    value = Column(Float)

    # bring these in instead of Measurement
    type = Column(String(50))
    units = Column(String(50))

    # This should be the foreign key
    # instrument = Column(String(50))
    # Link the instrument id with a foreign key
    instrument_id = Column(Integer, ForeignKey('instrument.id'))
    # Link the Instrument class
    instrument = relationship('Instrument')

    # id is a mapped column for many-to-many with observers
    id: Mapped[int] = mapped_column(primary_key=True)
    observers: Mapped[List[Observer]] = relationship(secondary=point_observers)
    



In [5]:
class PointsMeasurementsPrototype(PointMeasurements):
    """
    Extend the points api class for the new data model
    """
    DB_NAME = "snow:hackweek@35.86.115.212/snowex"
    MODEL = PointDataPrototype

## Example of adding a new point, this doens't work yet

In [6]:
def add_entry(instrument_name, observer_names, **kwargs):
    # Check if the instrument already exists
    instrument = session.query(Instrument).filter_by(name=instrument_name).first()
    # observer = session.query(Observer).filter_by(name=observer_name).first()
    
    if not instrument:
        # If the instrument does not exist, create it
        instrument = Instrument(name=instrument_name)
        session.add(instrument)
        session.commit()  # Commit to ensure instrument is saved and has an ID

    observer_list = []
    for obs_name in observer_names:
        observer = session.query(Observer).filter_by(last_name=observer_name).first()
        if not observer:
            # If the instrument does not exist, create it
            instrument = Observer(last_name=instrument_name)
            session.add(observer)
            session.commit()  # Commit to ensure instrument is saved and has an ID
        observer_list.append(observer)
        
    
    # Now that the instrument exists, create the entry, notice we only need the instrument object
    new_entry = PointDataPrototype(**kwargs, instrument=instrument, observers=observer_list)
    session.add(new_entry)
    session.commit()